In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('./data/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
onehot_encoded_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoded_geo.fit_transform(data[['Geography']]).toarray()

In [6]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoded_geo.get_feature_names_out(['Geography']))

In [8]:
data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
X = data.drop('EstimatedSalary', axis = 1)
y = data['EstimatedSalary']

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=42)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
## ANN Regression

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1) # Linear AF for regression
])

C:\ProgramData\anaconda3\envs\tf-nn\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.compile(
    optimizer='adam',
    loss = 'mean_absolute_error',
    metrics = ['mae']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,585 (14.00 KB)

 Trainable params: 3,585 (14.00 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [22]:
log_dir = 'regressionlogs/fit'
tensorboard_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [23]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [24]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs = 100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 99444.5391 - mae: 99444.5391 - val_loss: 92672.9219 - val_mae: 92672.9219
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 68135.8750 - mae: 68135.8750 - val_loss: 50815.9297 - val_mae: 50815.9297
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50149.0859 - mae: 50149.0859 - val_loss: 50509.8945 - val_mae: 50509.8945
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step - loss: 49938.5352 - mae: 49938.5352 - val_loss: 50465.2227 - val_mae: 50465.2227
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - loss: 49846.5938 - mae: 49846.5938 - val_loss: 50389.5469 - val_mae: 50389.5469
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - loss: 49755.7109 - mae: 49755.7109 - val_loss: 50558.3789 - val_mae: 50558.3789
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 49747.1133 - mae: 49747.1133 - val_loss: 50356.5547 - val_mae: 50356.5547
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 999us

In [25]:
#Eval the model

In [26]:
%load_ext tensorboard

In [28]:
%tensorboard --logdir regressionlogs/fit

In [29]:
test_loss, test_mae = model.evaluate(X_test, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step - loss: 50315.2617 - mae: 50315.2617
